In [1]:
import dask.dataframe as dd

# Carregar os dados de vendas com calendário
sales_calendar = dd.read_parquet('../data/merged_sales_with_calendar_parquet/')

# Carregar os dados de preços
item_prices = dd.read_parquet('../data/item_prices_filled_parquet/')

In [2]:
# Converter yearweek para string nos dois DataFrames
sales_calendar['yearweek'] = sales_calendar['yearweek'].astype(str)
item_prices['yearweek'] = item_prices['yearweek'].astype(str)

In [3]:
# Fazer o merge dos DataFrames usando 'item', 'store_code', e 'yearweek' como chave
merged_data = sales_calendar.merge(item_prices, on=['item', 'store_code', 'yearweek'], how='left')

# Verificar as primeiras linhas do DataFrame resultante
print(merged_data.head())


                          id                 item     category_x  \
0  HOME_&_GARDEN_2_437_NYC_2  HOME_&_GARDEN_2_437  HOME_&_GARDEN   
1  HOME_&_GARDEN_2_437_NYC_2  HOME_&_GARDEN_2_437  HOME_&_GARDEN   
2    SUPERMARKET_1_052_NYC_2    SUPERMARKET_1_052    SUPERMARKET   
3    SUPERMARKET_1_052_NYC_2    SUPERMARKET_1_052    SUPERMARKET   
4    SUPERMARKET_1_053_NYC_2    SUPERMARKET_1_053    SUPERMARKET   

        department   store store_code    region       d  sales yearweek  \
0  HOME_&_GARDEN_2  Harlem      NYC_2  New York  d_1053      0   201351   
1  HOME_&_GARDEN_2  Harlem      NYC_2  New York  d_1053      0   201351   
2    SUPERMARKET_1  Harlem      NYC_2  New York  d_1053      1   201351   
3    SUPERMARKET_1  Harlem      NYC_2  New York  d_1053      1   201351   
4    SUPERMARKET_1  Harlem      NYC_2  New York  d_1053      0   201351   

        date event     category_y  sell_price  
0 2013-12-16  <NA>  HOME_&_GARDEN      9.9625  
1 2013-12-16  <NA>  HOME_&_GARDEN      9.962

In [4]:
import os
import gc

# Liberar objetos não utilizados e memória
gc.collect()

# Definir o caminho do diretório
output_dir = '../data/merged_sales_with_calendar_and_prices'

# Criar o diretório se ele não existir
os.makedirs(output_dir, exist_ok=True)

# Aumentar o número de partições se o arquivo ainda for muito grande
merged_data_repartitioned = merged_data.repartition(npartitions=15)

# Salvar as partições uma por vez (removendo 'write_index')
for i, partition in enumerate(merged_data_repartitioned.to_delayed()):
    partition.compute().to_parquet(f'{output_dir}/part_{i}.parquet')

In [ ]:
# Verificar o tamanho de cada partição
print(merged_data_repartitioned.map_partitions(len).compute())